# Задание:
### Нужно написать код на питоне который решает задачу коммивояжера методом ветвей и границ, а также проверяет граф на связность и существует ли в графе гамильтонов цикл.  Проверить его на 10 сгенерированных графов(не полных, связных) и решить данную задачу для Фунзенского района города Владивосток.

In [1]:
import pandas as pd
import numpy as np
import re
import time
import networkx as nx
from itertools import combinations
import random
from tqdm import tqdm
from networkx.convert_matrix import from_numpy_array

# Долгая проверка: будет идти более 20 часов т.к. проверяет 100 вершин с шагом 10.

In [12]:
# Определение класса Node, который будет использоваться для представления узлов в дереве ветвей и границ.
class Node:
    def __init__(self, level, path):
        self.level = level  # уровень узла в дереве
        self.path = path  # путь от корня до этого узла
        self.bound = 0.0  # граница (см. функцию get_bound)

# Функция для расчета границы для узла u.
# Граница - это нижняя оценка стоимости пути, который начинается в корне, проходит через u и затем проходит через оставшиеся узлы.
def get_bound(u, n, graph):
    u.bound = sum([min(graph[i]) for i in range(n) if i not in u.path])

# Основная функция для решения задачи коммивояжера.
def TSP(graph):
    n = len(graph)
    result_path = []
    min_length = float('inf')

    # Инициализация корневого узла и вычисление его границы
    root = Node(0, [0])
    get_bound(root, n, graph)

    # Очередь для хранения "живых" узлов
    Q = [root]

    # Основной цикл: продолжается до тех пор, пока есть "живые" узлы
    while Q:
        # Выбираем узел с минимальной границей
        min_node = min(Q, key=lambda node: node.bound)
        Q.remove(min_node)

        # Если все вершины включены в путь
        if min_node.bound < min_length and min_node.level == n-1:
            min_node.path.append(min_node.path[0])  # добавить начальную вершину в конец пути для формирования цикла
            # Проверка, является ли текущий путь минимальным
            if sum(graph[min_node.path[i-1]][min_node.path[i]] for i in range(n)) < min_length:
                min_length = sum(graph[min_node.path[i-1]][min_node.path[i]] for i in range(n))
                result_path = min_node.path
        # Если все вершины не включены, сгенерировать потомков
        elif min_node.bound < min_length:
            for i in range(n):
                if i not in min_node.path:
                    child = Node(min_node.level+1, min_node.path[:])
                    child.path.append(i)
                    get_bound(child, n, graph)
                    Q.append(child)  # добавить потомка в очередь
    return result_path, min_length

# Функция для проверки связности графа
def check_connected(graph):
    G = nx.from_numpy_array(np.array(graph))
    return nx.is_connected(G)

# Функция для проверки наличия гамильтонова цикла в графе
def check_hamiltonian(graph):
    G = nx.from_numpy_array(np.array(graph))
    try:
        cycle = nx.algorithms.tournament.hamiltonian_path(G)
        return True
    except:
        return False

# Функция для генерации связного графа
def generate_connected_graph(vertices):
    graph = [[0 for _ in range(vertices)] for _ in range(vertices)]
    edges = list(combinations(range(vertices), 2))
    random.shuffle(edges)

    # Гарантируем связность графа
    for i in range(vertices-1):
        graph[edges[i][0]][edges[i][1]] = random.randint(1, 100)
        graph[edges[i][1]][edges[i][0]] = graph[edges[i][0]][edges[i][1]]

    # Добавляем случайные ребра
    for edge in edges[vertices-1:]:
        if random.random() < 0.6:  # вероятность создания ребра
            graph[edge[0]][edge[1]] = random.randint(1, 100)
            graph[edge[1]][edge[0]] = graph[edge[0]][edge[1]]
    return graph

# Генерация случайных графов и проверка их связности и наличия гамильтонова цикла
for size in tqdm(range(10, 101, 10)):
    for _ in range(10):
        graph = generate_connected_graph(size)
        start_time = time.time()
        print('Graph:\n', np.array(graph))
        print('Connected:', check_connected(graph))
        print('Has Hamiltonian cycle:', check_hamiltonian(graph))
        path, length = TSP(graph)
        end_time = time.time()
        print('TSP Path:', path)
        print('TSP Path length:', length)
        print('Execution time:', end_time - start_time, 'seconds\n')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Graph:
 [[ 0  0  0  8  0 19  0  0  0 41]
 [ 0  0 47 36  0 98  0 74  4  5]
 [ 0 47  0 75 89  0 52 52 70  0]
 [ 8 36 75  0 25 16 59 46 40 28]
 [ 0  0 89 25  0  9  3 96  0 95]
 [19 98  0 16  9  0 67 57 52 83]
 [ 0  0 52 59  3 67  0  0 78  6]
 [ 0 74 52 46 96 57  0  0  7 67]
 [ 0  4 70 40  0 52 78  7  0  0]
 [41  5  0 28 95 83  6 67  0  0]]
Connected: True
Has Hamiltonian cycle: False


  0%|                                                                                           | 0/10 [00:05<?, ?it/s]


KeyboardInterrupt: 

# Быстрая проверка. Проверяется при помощи 1 матрицы размерностью 10х10.

In [13]:
class Node:
    def __init__(self, level, path):
        self.level = level  
        self.path = path  
        self.bound = 0.0  

def get_bound(u, n, graph):
    u.bound = sum([min(graph[i][graph[i] != np.inf]) if isinstance(graph[i][graph[i] != np.inf], list) else graph[i][graph[i] != np.inf] for i in range(n) if i not in u.path])

def TSP(graph):
    n = len(graph)  
    result_path = []  
    min_length = float('inf')  

    root = Node(0, [0])
    get_bound(root, n, graph)  

    Q = [root]  

    while Q:
        min_node = min(Q, key=lambda node: node.bound)
        Q.remove(min_node)  

        if min_node.bound < min_length and min_node.level == n - 1:
            min_node.path.append(min_node.path[0])  
            if sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n)) < min_length:
                min_length = sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n))
                result_path = min_node.path
        elif min_node.bound < min_length:
            for i in range(n):
                if i not in min_node.path:
                    child = Node(min_node.level + 1, min_node.path[:])
                    child.path.append(i)
                    get_bound(child, n, graph)
                    Q.append(child)
    return result_path, min_length

def check_connected(graph):
    G = nx.from_numpy_array(graph)  
    return nx.is_connected(G)

def hamilton(G):
    F = [(G,[G[0]])]
    n = len(G)
    while F:
        graph,path = F.pop()
        confs = [(graph[:i] + graph[i+1:], path + [graph[i]]) for i in range(len(graph)) if graph[i] != path[0] and graph[i] not in path[1:]]
        for g,p in confs:
            if len(p) == n:
                return p
            else:
                F.append((g,p))
    return []

np.random.seed(42)  # for reproducibility
graph = np.random.randint(1, 1000, size=(10, 10))  # generate a 10x10 random cost matrix
np.fill_diagonal(graph, 0)  # set diagonal to 0

start_time = time.time()
print('Graph:\n', graph)
print('Connected:', check_connected(graph))
print('Has Hamiltonian cycle:', bool(hamilton(graph.tolist())))
path, length = TSP(graph.tolist())
end_time = time.time()

print('TSP Path:', path)
print('TSP Path length:', length)
print('Execution time:', end_time - start_time, 'seconds\n')
print('The actual route distance:', sum(graph[path[i - 1]][path[i]] for i in range(len(path))))

Graph:
 [[  0 436 861 271 107  72 701  21 615 122]
 [467   0 331 459  88 373 100 872 664 131]
 [662 309   0 344 492 414 806 386 192 956]
 [277 161 460   0  22 253 748 857 561 475]
 [ 59 511 682 476   0 976 783 190 958 687]
 [958 563 876 567 244   0 505 131 485 819]
 [647  21 841 167 274 388   0 316  14 242]
 [777 346 565 898 340  92 367   0 455 428]
 [509 776 943  35 206  81 932 562   0 388]
 [  2 390 566 106 772 822 477 703 402   0]]
Connected: True
Has Hamiltonian cycle: True
TSP Path: [0, 8, 5, 7, 6, 1, 9, 3, 4, 2, 0]
TSP Path length: 2156
Execution time: 0.35381603240966797 seconds

The actual route distance: 2818


Этот код решает задачу коммивояжера с помощью метода ветвей и границ. Код предполагает, что граф является **связным**, но он **не обязан быть полным**. В задаче коммивояжера речь идет о нахождении самого короткого возможного маршрута, который посещает каждый город и возвращается в исходный город, так что связность графа - ключевое требование.

Согласно коду, путь строится так, что каждый следующий город (узел), выбирается на основе его "bound", который представляет собой оценку расстояния, которое должно быть пройдено для достижения этого узла. По мере продвижения по графу, узлы с меньшими оценками "bound" имеют больший приоритет для посещения.

Вызов функции `check_connected` проверяет связность графа. Если граф связный, то `nx.is_connected(G)` вернет `True`. Это важно, потому что если граф не связный, то не может быть найден Гамильтонов цикл.

Функция `hamilton` проверяет наличие Гамильтонова цикла в графе. Это делается путем создания стека (F) с начальными конфигурациями графа (G) и путями, а затем добавления и удаления конфигураций и путей до тех пор, пока не будет найден Гамильтонов цикл или пока не кончатся конфигурации.

В конце кода генерируется случайный связный граф, замеряется время исполнения алгоритма и выводится самый короткий путь и его длина.

Чтобы вместо случайного графа использовать данные из файла Excel, вы должны прочитать данные из файла 'vershini.xlsx' вместо генерации случайного графа. Вот как вы можете сделать это:

In [ ]:
# df = pd.read_excel('vershini.xlsx', header=None) # загружаем данные без заголовка
# graph = df.values # извлекаем значения из dataframe и создаем матрицу смежности

Эти строки нужно вставить и раскоментить в коде выше вместо:

In [ ]:
# np.random.seed(42)  # for reproducibility
# graph = np.random.randint(1, 1000, size=(10, 10))  # generate a 10x10 random cost matrix
# np.fill_diagonal(graph, 0)  # set diagonal to 0

# Основной код, который считывает информацию с файла.

In [14]:
def hamilton(G):
    F = [(G,[G[0]])] # Создаём стек (F), содержащий начальные конфигурации графа (G) и пути
    n = len(G) # Получаем размерность графа (количество вершин)
    while F: # Пока в стеке есть элементы
        graph, path = F.pop() # Берем верхний элемент стека: граф и путь
        confs = [(graph[:i] + graph[i+1:], path + [graph[i]])  # Создаём новые конфигурации графа
                 for i in range(len(graph)) 
                 if graph[i] not in path] # Проверяем, не содержится ли следующая вершина в текущем пути
        for g, p in confs: # Для каждой новой конфигурации
            if len(p) == n: # Если путь содержит все вершины
                return p # Возвращаем этот путь
            else:
                F.append((g,p)) # Иначе добавляем эту конфигурацию в стек
    return None # Если нет Гамильтонова цикла, возвращаем None

class Node: # Класс для узла в ветвлении и ограничении
    def __init__(self, level, path):
        self.level = level # Уровень узла в дереве ветвления и ограничения
        self.path = path # Пройденный путь до узла
        self.bound = 0.0 # Нижняя граница стоимости пути до узла

def get_bound(u, n, graph): # Функция для расчета нижней границы стоимости пути до узла
    u.bound = sum([min(graph[i][graph[i] != np.inf]) if isinstance(graph[i][graph[i] != np.inf], list) else graph[i][graph[i] != np.inf] for i in range(n) if i not in u.path])

def TSP(graph): # Главная функция для решения задачи коммивояжера
    n = len(graph) # Получаем размерность графа
    result_path = [] # Инициализируем результативный путь
    min_length = float('inf') # Инициализируем минимальную длину пути бесконечностью

    root = Node(0, [0]) # Создаем корневой узел
    get_bound(root, n, graph) # Рассчитываем нижнюю границу для корневого узла

    Q = [root] # Инициализируем очередь узлов

    while Q: # Пока в очереди есть узлы
        min_node = min(Q, key=lambda node: node.bound) # Берем узел с минимальной нижней границей
        Q.remove(min_node) # Удаляем этот узел из очереди

        if min_node.bound < min_length and min_node.level == n - 1: # Если мы достигли конца и нашли более короткий путь
            min_node.path.append(min_node.path[0]) # Добавляем в путь начальную вершину, чтобы создать цикл
            if sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n)) < min_length: # Если новый путь короче
                min_length = sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n)) # Обновляем минимальную длину
                result_path = min_node.path # Обновляем результативный путь
        elif min_node.bound < min_length: # Если узел имеет потенциал для поиска более короткого пути
            for i in range(n): # Для каждой вершины графа
                if i not in min_node.path: # Если вершина еще не была посещена
                    child = Node(min_node.level + 1, min_node.path[:]) # Создаем дочерний узел
                    child.path.append(i) # Добавляем новую вершину в путь
                    get_bound(child, n, graph) # Рассчитываем нижнюю границу для дочернего узла
                    Q.append(child) # Добавляем дочерний узел в очередь
    return result_path, min_length # Возвращаем результативный путь и его длину

def check_connected(graph): # Функция для проверки связности графа
    G = nx.from_numpy_array(graph)
    return nx.is_connected(G)

df = pd.read_excel('filename.xlsx')

n = max([int(re.findall(r'\d+', s)[1]) for s in df['расстояние между перекрестками'].values]) + 1

graph = np.full((n, n), np.inf)

for idx, row in df.iterrows():
    i, j = map(int, re.findall(r'\d+', row['расстояние между перекрестками']))
    if pd.isnull(row['расстояние в метрах']):
        distance = np.inf
    else:
        distance = row['расстояние в метрах']
        if isinstance(distance, str):
            distance = float(distance.replace(',', '.'))
    graph[i, j] = distance
    graph[j, i] = distance

np.fill_diagonal(graph, 0)

start_time = time.time()
print('Graph:\n', graph)
print('Connected:', check_connected(graph))
print('Has Hamiltonian cycle:', bool(hamilton(graph.tolist())))
path, length = TSP(graph.tolist())
end_time = time.time()

print('TSP Path:', path)
print('TSP Path length:', length)
print('Execution time:', end_time - start_time, 'seconds')
print('The actual route distance:', sum(graph[path[i - 1]][path[i]] for i in range(len(path))))

Graph:
 [[  0.    inf   inf ...   inf   inf   inf]
 [  inf   0.  456.3 ...   inf   inf   inf]
 [  inf 456.3   0.  ...   inf   inf   inf]
 ...
 [  inf   inf   inf ...   0.    inf   inf]
 [  inf   inf   inf ...   inf   0.    inf]
 [  inf   inf   inf ...   inf   inf   0. ]]
Connected: True
Has Hamiltonian cycle: True
TSP Path: []
TSP Path length: inf
Execution time: 15.848985195159912 seconds
The actual route distance: 0


Кажется, что граф не имеет цикла путешественника продавца (TSP), что и вызывает результат TSP Path: [] и TSP Path length: inf. Это может быть связано с тем, что граф не полностью связный или в графе присутствуют вершины, доступные только из одного направления.

Тем не менее, код отрабатывает корректно в том смысле, что он выполняет задачу поиска пути TSP для данного графа. 

`Цикл путешественника-продавца (TSP, Travelling Salesman Problem) – это задача, в которой необходимо найти самый короткий возможный цикл, который проходит через каждую вершину в графе ровно один раз и возвращается обратно в исходную вершину. Это классическая задача комбинаторной оптимизации, широко известная в теории вычислительной сложности и теории графов.В данном контексте "цикл" относится к пути в графе, который начинается и заканчивается в одной и той же вершине и посещает каждую вершину ровно один раз. Другими словами, это путь, который образует "цикл" по всем вершинам графа без повторения вершин.`


Стоит проверить нет ли ошибок в данных, таких как вершины, доступные только из одного направления, или вершины, которые не связаны ни с одной другой вершиной.

In [ ]:
def hamilton(G):
    F = [(G,[G[0]])] # Создаём стек (F), содержащий начальные конфигурации графа (G) и пути
    n = len(G) # Получаем размерность графа (количество вершин)
    while F: # Пока в стеке есть элементы
        graph, path = F.pop() # Берем верхний элемент стека: граф и путь
        confs = [(graph[:i] + graph[i+1:], path + [graph[i]])  # Создаём новые конфигурации графа
                 for i in range(len(graph)) 
                 if graph[i] not in path] # Проверяем, не содержится ли следующая вершина в текущем пути
        for g, p in confs: # Для каждой новой конфигурации
            if len(p) == n: # Если путь содержит все вершины
                return p # Возвращаем этот путь
    return None # Если нет Гамильтонова цикла, возвращаем None

class Node: # Класс для узла в ветвлении и ограничении
    def __init__(self, level, path):
        self.level = level # Уровень узла в дереве ветвления и ограничения
        self.path = path # Пройденный путь до узла
        self.bound = 0.0 # Нижняя граница стоимости пути до узла

def get_bound(u, n, graph): # Функция для расчета нижней границы стоимости пути до узла
    u.bound = sum([min(graph[i][graph[i] != np.inf]) if isinstance(graph[i][graph[i] != np.inf], list) else graph[i][graph[i] != np.inf] for i in range(n) if i not in u.path])

def TSP(graph): # Главная функция для решения задачи коммивояжера
    n = len(graph) # Получаем размерность графа
    result_path = [] # Инициализируем результативный путь
    min_length = float('inf') # Инициализируем минимальную длину пути бесконечностью

    root = Node(0, [0]) # Создаем корневой узел
    get_bound(root, n, graph) # Рассчитываем нижнюю границу для корневого узла

    Q = [root] # Инициализируем очередь узлов

    while Q: # Пока в очереди есть узлы
        min_node = min(Q, key=lambda node: node.bound) # Берем узел с минимальной нижней границей
        Q.remove(min_node) # Удаляем этот узел из очереди

        if min_node.bound < min_length and min_node.level == n - 1: # Если мы достигли конца и нашли более короткий путь
            min_node.path.append(min_node.path[0]) # Добавляем в путь начальную вершину, чтобы создать цикл
            if sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n)) < min_length: # Если новый путь короче
                min_length = sum(graph[min_node.path[i - 1]][min_node.path[i]] for i in range(n)) # Обновляем минимальную длину
                result_path = min_node.path # Обновляем результативный путь
        elif min_node.bound < min_length: # Если узел имеет потенциал для поиска более короткого пути
            for i in range(n): # Для каждой вершины графа
                if i not in min_node.path: # Если вершина еще не была посещена
                    child = Node(min_node.level + 1, min_node.path[:]) # Создаем дочерний узел
                    child.path.append(i) # Добавляем новую вершину в путь
                    get_bound(child, n, graph) # Рассчитываем нижнюю границу для дочернего узла
                    Q.append(child) # Добавляем дочерний узел в очередь
    return result_path, min_length # Возвращаем результативный путь и его длину

def check_connected(graph): # Функция для проверки связности графа
    G = nx.from_numpy_array(graph)
    return nx.is_connected(G)

df = pd.read_excel('vershini.xlsx', header=None) # загружаем данные без заголовка

graph = df.values # извлекаем значения из dataframe и создаем матрицу смежности

start_time = time.time()
print('Graph:\n', graph)
print('Connected:', check_connected(graph))
print('Has Hamiltonian cycle:', bool(hamilton(graph.tolist())))
path, length = TSP(graph.tolist())
end_time = time.time()

print('TSP Path:', path)
print('TSP Path length:', length)
print('Execution time:', end_time - start_time, 'seconds')
print('The actual route distance:', sum(graph[path[i - 1]][path[i]] for i in range(len(path))))

Graph:
 [[   0.          126.13623183  163.29908977  791.28820252  152.88568158
  1262.08585825  114.90831446 2099.7261034   889.1376664  2124.220146
   756.89612319   60.32983407 1366.8011019   114.0786188  1240.08539083
   499.90394339  754.14783949 1705.51977942  800.48989557 2001.86833723]
 [ 126.13623183    0.           37.16312346  909.62253484   81.80177773
  1384.75824053   80.38484835 2224.58626925  942.65396955 2129.54479785
   882.97120268  146.87893657 1418.51616034   53.74825602 1282.88905702
   613.24504169  875.41579572 1695.24196496  920.5141762  2085.11046306]
 [ 163.29908977   37.16312346    0.          944.81977116   88.4232405
  1421.04914976  102.5709648  2261.3773491   961.13365453 2132.64062969
   920.12358334  181.50686751 1435.6255785    77.39432981 1297.68995103
   647.5988323   911.35575367 1694.11164595  956.13040848 2110.53577472]
 [ 791.28820252  909.62253484  944.81977116    0.          889.55565669
   773.1463337   853.90672686 1342.04514304 1191.3766284